In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Generating example EOD data
np.random.seed(42)
date_rng = pd.date_range(start='2023-01-01', end='2023-11-22', freq='B')  # Business days
eod_data = pd.DataFrame({
    'Date': date_rng,
    'Close': np.cumprod(1 + np.random.normal(0, 0.01, size=len(date_rng))) * 100
})
eod_data.set_index('Date', inplace=True)

# 1. Daily Returns
eod_data['Daily_Return'] = eod_data['Close'].pct_change() * 100

# 2. Cumulative Returns
eod_data['Cumulative_Return'] = ((1 + eod_data['Daily_Return'] / 100).cumprod() - 1) * 100
 
# 3. Average Daily Returns
average_daily_return = eod_data['Daily_Return'].mean()

# 4. Volatility
volatility = eod_data['Daily_Return'].std()

# 5. Moving Averages
eod_data['50_Day_MA'] = eod_data['Close'].rolling(window=50).mean()
eod_data['200_Day_MA'] = eod_data['Close'].rolling(window=200).mean()

# 6. Relative Strength Index (RSI)
delta = eod_data['Close'].diff(1)
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)

average_gain = gain.rolling(window=14).mean()
average_loss = loss.rolling(window=14).mean()

rs = average_gain / average_loss
eod_data['RSI'] = 100 - (100 / (1 + rs))

# 7. Sharpe Ratio
sharpe_ratio = (average_daily_return / volatility) * np.sqrt(len(eod_data))

# 8. Maximum Drawdown
cumulative_returns = (1 + eod_data['Daily_Return'] / 100).cumprod()
peak_value = cumulative_returns.cummax()
drawdown = (cumulative_returns - peak_value) / peak_value
max_drawdown = drawdown.min() * 100

# Displaying the results
print(f"Average Daily Return: {average_daily_return:.4f}%")
print(f"Volatility: {volatility:.4f}%")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")
print(f"Maximum Drawdown: {max_drawdown:.4f}%")

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(eod_data['Close'], label='AAPL Close Price')
plt.plot(eod_data['50_Day_MA'], label='50-Day MA')
plt.plot(eod_data['200_Day_MA'], label='200-Day MA')
plt.legend()
plt.title('AAPL Stock Price and Moving Averages')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()
